# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

In [1]:
# Imports
import os
import warnings
import tools
import models as c_models
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Root CSV files directory
dirname = "./data/"

# Constant frame count.
frames = 100

## Preparation Stage
### Load data and normalize
For training it's required to extend/reduce every dataset to n frames, where n is `frames`.

In [2]:
listfile = os.listdir(dirname)
contents = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        contents.append((wordname, content))
data = contents

In [3]:
# Print the content of the first dataframe
contents[0][1]

,face_x,face_y,landmark_x_1,landmark_y_1,landmark_x_2,landmark_y_2,landmark_x_3,landmark_y_3,landmark_x_4,landmark_y_4,...,landmark_x_38,landmark_y_38,landmark_x_39,landmark_y_39,landmark_x_40,landmark_y_40,landmark_x_41,landmark_y_41,landmark_x_42,landmark_y_42
0,0.470624,0.313346,0.639333,0.944016,0.588136,0.923641,0.561776,0.890851,0.577628,0.867982,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.470841,0.318451,0.351937,0.960826,0.411960,0.935828,0.435300,0.886520,0.435451,0.847356,...,0.619671,0.821530,0.683979,0.790299,0.677252,0.774806,0.654642,0.813162,0.638180,0.845178
2,0.469118,0.320056,0.365665,0.956447,0.420067,0.908137,0.447947,0.849639,0.436687,0.799112,...,0.605166,0.822862,0.669363,0.795220,0.663645,0.791355,0.640720,0.825665,0.622981,0.855132
3,0.469637,0.320771,0.380982,0.957185,0.429653,0.899122,0.453137,0.833393,0.440341,0.774351,...,0.603590,0.823892,0.661824,0.792022,0.660768,0.776878,0.641793,0.811655,0.624695,0.842568
4,0.471696,0.319425,0.387040,0.958757,0.441715,0.887443,0.464157,0.826509,0.456501,0.775087,...,0.597416,0.806157,0.656806,0.783026,0.648856,0.743433,0.629925,0.785155,0.613204,0.822693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
96,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
97,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Split data
Split the dataset up into the following segments:
1. Training Data: 60%
2. Validation Data: 20%
3. Test Data: 20%


In [4]:
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.40, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.50, random_state=42)

In [5]:
# Display data distribution
print("Total:", len(labels))
print("Training:", len(y_train), len(y_train) / len(labels) * 100)
print("Val:", len(y_val), len(y_val) / len(labels) * 100)
print("Test:", len(y_test), len(y_test) / len(labels) * 100)

Total: 347
Training: 208 59.94236311239193
Val: 69 19.88472622478386
Test: 70 20.172910662824208


### Tokenize (One Hot)

In [6]:
import tools
tokenizer = tools.tokenize(dirname)
print(tokenizer.word_index)
encoded_train=tokenizer.texts_to_sequences([y_train])[0]
encoded_val=tokenizer.texts_to_sequences([y_val])[0]
encoded_test=tokenizer.texts_to_sequences([y_test])[0]
y_train = to_categorical(encoded_train)
y_val = to_categorical(encoded_val)
y_test = to_categorical(encoded_test)
print(y_train)

{'welt': 1, 'deutschland': 2, 'hallo': 3, 'computer': 4}
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


In [7]:
# Making numpy arrays
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [8]:
print(y_train)

[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


## Training Stage
Configure the model and train it.

Metrics:
![accuracy](assets/accuracy.png)
![precision and recall formula](assets/precision_recall_formula.png)
![precision and recall](assets/precision_recall.png)


### Configure

In [9]:
model = Sequential()
model.add(layers.LSTM(64, return_sequences=True,
               input_shape=(100, 86)))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32))  # return a single vector of dimension 32
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

### Train

In [10]:
history=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_val,y_val))

Train on 208 samples, validate on 69 samples
Epoch 1/100
208/208 [==============================] - 14s 67ms/sample - loss: 1.5135 - accuracy: 0.2596 - precision: 0.9000 - recall: 0.0433 - val_loss: 1.3697 - val_accuracy: 0.3478 - val_precision: 1.0000 - val_recall: 0.0725
Epoch 2/100
208/208 [==============================] - 4s 17ms/sample - loss: 1.3695 - accuracy: 0.3365 - precision: 0.6154 - recall: 0.0769 - val_loss: 1.2711 - val_accuracy: 0.4638 - val_precision: 1.0000 - val_recall: 0.0870
Epoch 3/100
208/208 [==============================] - 4s 17ms/sample - loss: 1.2083 - accuracy: 0.4471 - precision: 0.9706 - recall: 0.1587 - val_loss: 1.1183 - val_accuracy: 0.4493 - val_precision: 1.0000 - val_recall: 0.2174
Epoch 4/100
208/208 [==============================] - 4s 17ms/sample - loss: 1.1443 - accuracy: 0.3942 - precision: 0.8182 - recall: 0.1731 - val_loss: 1.0747 - val_accuracy: 0.4058 - val_precision: 0.8000 - val_recall: 0.2319
Epoch 5/100
208/208 [=====================

Epoch 37/100
208/208 [==============================] - 4s 19ms/sample - loss: 0.3589 - accuracy: 0.8894 - precision: 0.9050 - recall: 0.8702 - val_loss: 0.4602 - val_accuracy: 0.8841 - val_precision: 0.8971 - val_recall: 0.8841
Epoch 38/100
208/208 [==============================] - 4s 20ms/sample - loss: 0.6915 - accuracy: 0.7981 - precision: 0.8088 - recall: 0.7933 - val_loss: 0.4979 - val_accuracy: 0.8696 - val_precision: 0.9077 - val_recall: 0.8551
Epoch 39/100
208/208 [==============================] - 4s 19ms/sample - loss: 0.3455 - accuracy: 0.8654 - precision: 0.8768 - recall: 0.8558 - val_loss: 0.5373 - val_accuracy: 0.8841 - val_precision: 0.8955 - val_recall: 0.8696
Epoch 40/100
208/208 [==============================] - 4s 20ms/sample - loss: 0.5209 - accuracy: 0.8462 - precision: 0.8529 - recall: 0.8365 - val_loss: 0.7574 - val_accuracy: 0.7681 - val_precision: 0.7761 - val_recall: 0.7536
Epoch 41/100
208/208 [==============================] - 4s 20ms/sample - loss: 0.442

Epoch 73/100
208/208 [==============================] - 4s 19ms/sample - loss: 0.3279 - accuracy: 0.8702 - precision: 0.8955 - recall: 0.8654 - val_loss: 0.4131 - val_accuracy: 0.8696 - val_precision: 0.9048 - val_recall: 0.8261
Epoch 74/100
208/208 [==============================] - 4s 19ms/sample - loss: 0.2663 - accuracy: 0.9231 - precision: 0.9497 - recall: 0.9087 - val_loss: 0.5274 - val_accuracy: 0.8696 - val_precision: 0.8939 - val_recall: 0.8551
Epoch 75/100
208/208 [==============================] - 4s 19ms/sample - loss: 0.5044 - accuracy: 0.8413 - precision: 0.8614 - recall: 0.8365 - val_loss: 0.5655 - val_accuracy: 0.7826 - val_precision: 0.8438 - val_recall: 0.7826
Epoch 76/100
208/208 [==============================] - 4s 20ms/sample - loss: 0.2965 - accuracy: 0.8894 - precision: 0.9104 - recall: 0.8798 - val_loss: 0.4839 - val_accuracy: 0.8551 - val_precision: 0.8806 - val_recall: 0.8551
Epoch 77/100
208/208 [==============================] - 4s 20ms/sample - loss: 0.249

### Evaluate

In [11]:
model.evaluate(x=x_test, y=y_test, batch_size=32)

70/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.3344818920961448, 0.8857143, 0.89705884, 0.87142855]

### Save model

In [12]:
model.save("sign_lang_recognition.h5")